# Data Cleaning Notebook

In [110]:
# Import the required libraries
import pandas as pd
import numpy as np

# set up pandas to display floats in a more human friendly way
pd.options.display.float_format = '{:,.2f}'.format

### Original training data files downloaded from data source were split between values and labels. 
#### Step 1: Conduct data cleaning USING FUNCTIONS on the Training and Test Files
#### Step 2: Combine the Training values and labels into a single file for use in model training.

In [111]:
# TRAINING DATA
train_values_raw = pd.read_csv('../data/original/TrainingSetValues/4910797b-ee55-40a7-8668-10efd5c1b960.csv')
train_values_raw.head(3)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,"6,000.00",2011-03-14,Roman,1390,Roman,34.94,-9.86,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.00,2013-03-06,Grumeti,1399,GRUMETI,34.70,-2.15,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.00,2013-02-25,Lottery Club,686,World vision,37.46,-3.82,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe


In [112]:
# TEST DATA 
test_values_raw = pd.read_csv('../data/original/TestSetValues/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv')
test_values_raw.head(3)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.00,2013-02-04,Dmdd,1996,DMDD,35.29,-4.06,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.00,2013-02-04,Government Of Tanzania,1569,DWE,36.66,-3.31,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.00,2013-02-01,NaN,1567,NaN,34.77,-5.00,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other


## Column and Row Information
* 40 columns/features in the raw data TRAINING and TEST
* 59,400 rows in the TRAINING data
* 14,850 rows in the TEST data

In [113]:
train_labels_raw = pd.read_csv('../data/original/TrainingSetLabels/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv')
print(train_labels_raw.info())
train_labels_raw.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB
None


,id,status_group
0,69572,functional
1,8776,functional


## Original Data Column Descriptions

* id - Numeric identifer for the waterpoint
* amount_tsh - Total static head (amount water available to waterpoint)
* date_recorded - The date the row was entered
* funder - Who funded the well
* gps_height - Altitude of the well
* installer - Organization that installed the well
* longitude - GPS coordinate
* latitude - GPS coordinate
* wpt_name - Name of the waterpoint if there is one
* num_private -
* basin - Geographic water basin
* subvillage - Geographic location
* region - Geographic location, NOTE: Hierarchy is Region > LGA > Ward
* region_code - Geographic location (coded)
* district_code - Geographic location (coded)
* lga - Geographic location
* ward - Geographic location
* population - Population around the well
* public_meeting - True/False
* recorded_by - Group entering this row of data
* scheme_management - Who operates the waterpoint
* scheme_name - Who operates the waterpoint
* permit - If the waterpoint is permitted
* construction_year - Year the waterpoint was constructed
* extraction_type - The kind of extraction the waterpoint uses
* extraction_type_group - The kind of extraction the waterpoint uses
* extraction_type_class - The kind of extraction the waterpoint uses
* management - How the waterpoint is managed
* management_group - How the waterpoint is managed
* payment - What the water costs
* payment_type - What the water costs
* water_quality - The quality of the water
* quality_group - The quality of the water
* quantity - The quantity of water
* quantity_group - The quantity of water
* source - The source of the water
* source_type - The source of the water
* source_class - The source of the water
* waterpoint_type - The kind of waterpoint
* waterpoint_type_group - The kind of waterpoint



## Data cleaning steps:
* __Duplicate check__ 

* __Address Null Values (nans)__

* __Address Zeros in Numeric Values__

* __String type normalization__
 
* __New column/feature creation__


* __Note:__ No Data type conversions needed

In [114]:
# Functions for Duplicate checks 
def has_exact_dups(df):
    dups = df[df.duplicated()]
    return len(dups) > 0

def has_identifier_dups(df, col_name='id'):
    num_rows = df.shape[0]
    num_ids = len(df[col_name].unique())
    return num_ids != num_rows

In [115]:
#Dup checking
print(has_exact_dups(train_values_raw))
print(has_identifier_dups(train_values_raw))
print(has_exact_dups(test_values_raw))
print(has_identifier_dups(test_values_raw))

False
False
False
False


### There are no exact duplicate rows nor duplicate identifers in the Training or Test datasets


In [116]:
# Make a deep copy and do procesing on this dataframe moving forward
# Deep copy has own copy of data and index.
train_values_processed = train_values_raw.copy(deep=True)
test_values_processed = test_values_raw.copy(deep=True)

In [117]:
#Null handler functions
def get_null_stats(df, col_name):
    print(df[col_name].unique())
    print(len(df[col_name].unique()))
    print(df[col_name].isna().sum())


def handle_nulls_inplace(df, cols_to_fill):
    for item in cols_to_fill:
        for key, value in item.items():
            df[key].fillna(value, inplace=True)

### Handling Null Values
#### TRAINING and TEST columns with Null values:
* funder : set to 'unknown' when null
* installer : set to 'unknown' when null
* subvillage : set to 'unknown' when null
* public_meeting : set to True when null
    * Training dataset: only 5055 out of 59,400 were False. 51,011 out of 59,4000 were True. 
    * Test dataset: only 1291 out of 14,850 were False, 12,738 out of 14,850 were True.
    * As the vast majority of both Training and Test waterpoints have public_meeting of True, use True to replace all nulls.
* scheme_management : set to 'unknown' when null
* scheme_name : set to 'unknown' when null
* permit : set to True when nulls
    * Training dataset: 38,852 out of 59,4000 were True. 17,492 out of 59,4000 where False. 
    * Test dataset:  9754 out of 14,850 were True. 4359 out of 14,850 were False
    * As the majority of both Training and Test waterpoints have public_meeting of True, use True to replace all nulls.


In [118]:
cols_to_fill = [{'funder': 'unknown'}, {'installer': 'unknown'}, {'subvillage': 'unknown'}, {'public_meeting': True}, {'scheme_management': 'unknown'}, {'scheme_name': 'unknown'}, {'permit': True}]

handle_nulls_inplace(train_values_processed, cols_to_fill)
handle_nulls_inplace(test_values_processed, cols_to_fill)


### Handing Zeros in Numeric Columns
* No change needed
    * latitude : No zeros
    * region_code : No zeros
* Drop data
    * num_private : ~98 of Train,  DROP this COLUMN from Train and Test dataset
    * longitude : ~3% of of Train and ~3% of Test, DROP these ROWS
    * district_code : Very, very few (23 rows in Train and 4 in Test) have zeros. DROP these ROWS.

* Replace Zeros
    * construction_year : ~35% of Train and ~35% of Test - Update zeros with the Average Construction Year.

* Do nothing. These 0 values seem in line with data used on the Offical Tanzanian Water Point Mapping System (WPMS)  [http://wpm.maji.go.tz/]
    * amount_tsh : ~70% of Train were Zero: ??? Take the average of pumps of a similar type? 
    * gps_height : ~ 34% of Train were Zero: ???? Take the average of pumps with similar latitude?
    * population : ~36% of Train and X of Test -  ???


In [119]:
# Functions for handling Zeros in Numeric columns

def count_zeros(df, col_name):
    return df[df[col_name]==0][col_name].count()


In [120]:
numeric_col_names = ['amount_tsh', 'gps_height', 'latitude', 'longitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year']

for col_name in numeric_col_names:
    the_train_count = count_zeros(train_values_processed, col_name)
    the_test_count = count_zeros(test_values_processed, col_name)
    if(the_test_count + the_train_count > 0):
        print('TRAIN:', col_name, the_train_count)
        print('TEST:', col_name, the_test_count)

TRAIN: amount_tsh 41639
TEST: amount_tsh 10410
TRAIN: gps_height 20438
TEST: gps_height 5211
TRAIN: longitude 1812
TEST: longitude 457
TRAIN: num_private 58643
TEST: num_private 14656
TRAIN: district_code 23
TEST: district_code 4
TRAIN: population 21381
TEST: population 5453
TRAIN: construction_year 20709
TEST: construction_year 5260


In [121]:
# Drop the num_private COLUMN
train_values_processed.drop('num_private', axis=1, inplace=True)
test_values_processed.drop('num_private', axis=1, inplace=True)

In [122]:
# Drop the rows with 0 longitude
train_indices_long = train_values_processed[train_values_processed['longitude'] == 0 ].index
test_indices_long = test_values_processed[test_values_processed['longitude'] == 0 ].index
train_values_processed.drop(train_indices_long, inplace=True)
test_values_processed.drop(test_indices_long, inplace=True)

# drop the rows with 0 district_code
train_indices_distric_code = train_values_processed[train_values_processed['district_code'] == 0 ].index
test_indices_distric_code = test_values_processed[test_values_processed['district_code'] == 0 ].index
train_values_processed.drop(train_indices_distric_code, inplace=True)
test_values_processed.drop(test_indices_distric_code, inplace=True)

In [123]:
# Get the average construction year for TRAIN and TEST
known_const_year_rows = train_values_processed[train_values_processed['construction_year']>0]
avg_counstruction_year = int(known_const_year_rows['construction_year'].mean().round())
print(avg_counstruction_year)

test_known_const_year_rows = test_values_processed[test_values_processed['construction_year']>0]
test_avg_counstruction_year = int(test_known_const_year_rows['construction_year'].mean().round())
print(test_avg_counstruction_year)

1997
1997


In [124]:
# Set construction_year to the average construction year where that value is 0
test_values_processed['construction_year'] = test_values_processed.apply(lambda row: avg_counstruction_year if row['construction_year'] == 0 else row['construction_year'], axis=1)
test_values_processed['construction_year'] = test_values_processed.apply(lambda row: test_avg_counstruction_year if row['construction_year'] == 0 else row['construction_year'], axis=1)

### Normalize String columns - all to lower case

* funder - Who funded the well
* installer - Organization that installed the well
* wpt_name - Name of the waterpoint if there is one
* basin - Geographic water basin
* subvillage - Geographic location
* region - Geographic location
* lga - Geographic location
* ward - Geographic location
* recorded_by - Group entering this row of data
* scheme_management - Who operates the waterpoint
* scheme_name - Who operates the waterpoint
* extraction_type - The kind of extraction the waterpoint uses
* extraction_type_group - The kind of extraction the waterpoint uses
* extraction_type_class - The kind of extraction the waterpoint uses
* management - How the waterpoint is managed
* management_group - How the waterpoint is managed
* payment_type - What the water costs
* payment_type - What the water costs
* water_quality - The quality of the water
* quality_group - The quality of the water
* quantity - The quantity of water
* quantity_group - The quantity of water
* source - The source of the water
* source_type - The source of the water
* source_class - The source of the water
* waterpoint_type - The kind of waterpoint
* waterpoint_type_group - The kind of waterpoin


In [125]:
# Normalize String values function(s)
def normalize_strings(df, col_name):
     df[col_name] = df.apply(lambda row: row[col_name].lower(), axis=1)

In [126]:

string_col_names = ['funder', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 'recorded_by','scheme_management','scheme_name', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type' , 'source_class', 'waterpoint_type', 'waterpoint_type_group']

for col_name in string_col_names:
    normalize_strings(train_values_processed, col_name)
    normalize_strings(test_values_processed, col_name)


### New Columns/Feature creation
* recorded_year -  Pulling out the year from date_recorded
* waterpoint_age - Calculate as recorded_year - construction_year
* region_with_code - Combine region and region_code. There are more region_code values than region values
* recorded_good_quality - True if quality_group == 'good', False if  anything other than 'good'
* recorded_good_quantity - True if quanity_group == 'sufficient', False if anythign other than 'sufficient'

 

In [127]:
# Functions for creating new features
def get_recorded_year(recorded_date_string):
    year = 0
    date_segs = recorded_date_string.split('-')
    if((len(date_segs) == 3) & (len(date_segs[0]) == 4)):
        try:
            year = int(date_segs[0])
        except:
            print("Not a valid year format.") 
    return year


def get_waterpoint_age(recorded_year, constructed_year):
    age = 0
    is_logical_year = constructed_year > 0
    is_logical_age = recorded_year > constructed_year
    if (is_logical_year & is_logical_age):
        age = recorded_year - constructed_year
    return age


def get_region_name_and_code(region, region_code):
    return region + '_' + str(region_code)


def get_recorded_good_quality(quality_group):
    result = False
    if ('good' == quality_group):
        result = True
    return result


def get_recorded_good_quanity(quanity_group):
    result = False
    if ('enough' == quanity_group):
        result = True
    return result
        

In [128]:
# recorded_year
train_values_processed['recorded_year'] = train_values_processed.apply(lambda row: get_recorded_year(row['date_recorded']), axis=1)
test_values_processed['recorded_year'] = test_values_processed.apply(lambda row: get_recorded_year(row['date_recorded']), axis=1)

In [129]:
# waterpoint_age
train_values_processed['waterpoint_age'] = train_values_processed.apply(lambda row: get_waterpoint_age(row['recorded_year'],row['construction_year']), axis=1)
test_values_processed['waterpoint_age'] = test_values_processed.apply(lambda row: get_waterpoint_age(row['recorded_year'], row['construction_year']), axis=1)

In [130]:
# region_code_and_name
train_values_processed['region_code_and_name'] = train_values_processed.apply(lambda row: get_region_name_and_code(row['region'],row['region_code']), axis=1)
test_values_processed['region_code_and_name'] = test_values_processed.apply(lambda row: get_region_name_and_code(row['region'], row['region_code']), axis=1)

In [131]:
# recorded_good_quality
train_values_processed['recorded_good_quality'] = train_values_processed.apply(lambda row: get_recorded_good_quality(row['quality_group']), axis=1)
test_values_processed['recorded_good_quality'] = test_values_processed.apply(lambda row: get_recorded_good_quality(row['quality_group']), axis=1)

In [132]:
# recorded_good_quanity
train_values_processed['recorded_good_quantity'] = train_values_processed.apply(lambda row: get_recorded_good_quanity(row['quantity_group']), axis=1)
test_values_processed['recorded_good_quantity'] = test_values_processed.apply(lambda row: get_recorded_good_quanity(row['quantity_group']), axis=1)

## Final Prep
* Add the class labels to the TRAINING dataset only
* Save both processed TRAINING and TEST to file for use in EDA and Classifer Modeling


In [133]:
train_values_processed_and_labeled = pd.merge(train_values_processed, train_labels_raw, on='id')

In [134]:
train_values_processed_and_labeled.to_csv('../data/train_processed_labeled.csv', index=False)
test_values_processed.to_csv('../data/test_processed.csv', index=False)

In [135]:
train_values_processed_and_labeled.shape

(57565, 45)

In [136]:
test_values_processed.shape

(14389, 44)